In [33]:
from gensim.models import KeyedVectors

# 路径替换为你下载的txt文件路径
file_path = 'tencent-ailab-embedding-zh-d200-v0.2.0.txt'

print("正在加载词向量，这可能需要几分钟，请耐心等待...")

# limit参数：如果你内存不够（比如只有8G/16G），建议加上 limit=500000
# 这表示只加载最常用的前50万个词，通常足够覆盖省份和常用形容词
# 如果内存充足（32G以上），可以去掉 limit 参数加载全部
wv = KeyedVectors.load_word2vec_format(file_path, binary=False, limit=5000000)

print("加载完成！")

# 测试一下是否成功
test_word = '广东'
if test_word in wv:
    print(f"'{test_word}' 的向量主要维度前5位: {wv[test_word][:5]}")
else:
    print(f"'{test_word}' 不在词表中")

正在加载词向量，这可能需要几分钟，请耐心等待...
加载完成！
'广东' 的向量主要维度前5位: [-0.126287 -0.039829  0.131457 -0.100277  0.117989]


In [ ]:

province_definitions = {
    # --- 直辖市 ---
    '北京': ['北京', '京', '北京人'],
    '上海': ['上海', '沪', '上海人'],
    '天津': ['天津', '津', '天津人'],
    '重庆': ['重庆', '渝', '重庆人'],

    # --- 华北地区 ---
    '河北': ['河北', '冀', '河北人'],
    '山西': ['山西', '晋', '山西人'],
    '内蒙古': ['内蒙古', '蒙', '内蒙', '内蒙古人'], # "蒙"有时会有歧义，但在语境中通常可用

    # --- 东北地区 (既包含单个省，也包含"东北"整体概念) ---
    '东北': ['东北', '东北人'], # 作为一个文化整体
    '辽宁': ['辽宁', '辽', '辽宁人'],
    '吉林': ['吉林', '吉', '吉林人'], # 注意："吉"字可能有"吉利"的噪音，但在平均算法下可控
    '黑龙江': ['黑龙江', '黑', '黑龙江人'],

    # --- 华东地区 ---
    '江苏': ['江苏', '苏', '江苏人'],
    '浙江': ['浙江', '浙', '浙江人'],
    '安徽': ['安徽', '皖', '安徽人'],
    '福建': ['福建', '闽', '福建人'],
    '江西': ['江西', '赣', '江西人'],
    '山东': ['山东', '鲁', '山东人'],
    # 台湾在地理分区上常归入华东或单独列出
    '台湾': ['台湾', '台', '台湾人'],

    # --- 华中地区 ---
    '河南': ['河南', '豫', '河南人'],
    '湖北': ['湖北', '鄂', '湖北人'],
    '湖南': ['湖南', '湘', '湖南人'],

    # --- 华南地区 ---
    '广东': ['广东', '粤', '广东人'],
    '广西': ['广西', '桂', '广西人'],
    '海南': ['海南', '琼', '海南人'],
    '香港': ['香港', '港', '香港人'],
    '澳门': ['澳门', '澳', '澳门人'],

    # --- 西南地区 ---
    '四川': ['四川', '川', '蜀', '四川人'], # 包含双简称
    '贵州': ['贵州', '贵', '黔', '贵州人'], # 包含双简称
    '云南': ['云南', '云', '滇', '云南人'], # 包含双简称
    '西藏': ['西藏', '藏', '西藏人'],

    # --- 西北地区 ---
    '陕西': ['陕西', '陕', '秦', '陕西人'], # 包含双简称
    '甘肃': ['甘肃', '甘', '陇', '甘肃人'], # 包含双简称
    '青海': ['青海', '青', '青海人'],
    '宁夏': ['宁夏', '宁', '宁夏人'],
    '新疆': ['新疆', '新', '新疆人'] # "新"字可能有噪音，但"新疆人"很准
}

# -------------------------------------------------------------
# 辅助检查代码：打印一下看看有多少个区域
print(f"共定义了 {len(province_definitions)} 个区域/省份。")

### 1.计算不同省份 和 词汇的相似度

In [45]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from tqdm import tqdm  # 用于显示进度条


csv_path = '13428_2021_1607_MOESM1_ESM.csv'  # 你的文件名
try:
    df_words = pd.read_csv(csv_path)
    # 如果 CSV 没有表头，可以用 header=None，并指定列名
    # df_words = pd.read_csv(csv_path, header=None, names=['word'])
    
    # 确保主要是字符串，去除空值
    word_list = df_words['Word'].dropna().astype(str).tolist()
    print(f"原始词表大小: {len(word_list)}")
except Exception as e:
    print(f"读取CSV失败: {e}")
    # 用于测试的假数据
    word_list = ['井盖', '面食', '大葱', '细腻', '火锅', '挖掘机', '豪爽', '诈骗']

# ==============================================================================
# 2. 定义省份同义词组 (Synonym Dictionary)
# ==============================================================================
# 这种方法能让结果更鲁棒（Robust）
province_definitions = {
    # --- 直辖市 ---
    '北京': ['北京', '京', '北京人'],
    '上海': ['上海', '沪', '上海人'],
    '天津': ['天津', '津', '天津人'],
    '重庆': ['重庆', '渝', '重庆人'],

    # --- 华北地区 ---
    '河北': ['河北', '冀', '河北人'],
    '山西': ['山西', '晋', '山西人'],
    '内蒙古': ['内蒙古', '蒙', '内蒙', '内蒙古人'], # "蒙"有时会有歧义，但在语境中通常可用

    # --- 东北地区 (既包含单个省，也包含"东北"整体概念) ---
    '东北': ['东北', '东北人'], # 作为一个文化整体
    '辽宁': ['辽宁', '辽', '辽宁人'],
    '吉林': ['吉林', '吉', '吉林人'], # 注意："吉"字可能有"吉利"的噪音，但在平均算法下可控
    '黑龙江': ['黑龙江', '黑', '黑龙江人'],

    # --- 华东地区 ---
    '江苏': ['江苏', '苏', '江苏人'],
    '浙江': ['浙江', '浙', '浙江人'],
    '安徽': ['安徽', '皖', '安徽人'],
    '福建': ['福建', '闽', '福建人'],
    '江西': ['江西', '赣', '江西人'],
    '山东': ['山东', '鲁', '山东人'],
    # 台湾在地理分区上常归入华东或单独列出
    '台湾': ['台湾', '台', '台湾人'],

    # --- 华中地区 ---
    '河南': ['河南', '豫', '河南人'],
    '湖北': ['湖北', '鄂', '湖北人'],
    '湖南': ['湖南', '湘', '湖南人'],

    # --- 华南地区 ---
    '广东': ['广东', '粤', '广东人'],
    '广西': ['广西', '桂', '广西人'],
    '海南': ['海南', '琼', '海南人'],
    '香港': ['香港', '港', '香港人'],
    '澳门': ['澳门', '澳', '澳门人'],

    # --- 西南地区 ---
    '四川': ['四川', '川', '蜀', '四川人'], # 包含双简称
    '贵州': ['贵州', '贵', '黔', '贵州人'], # 包含双简称
    '云南': ['云南', '云', '滇', '云南人'], # 包含双简称
    '西藏': ['西藏', '藏', '西藏人'],

    # --- 西北地区 ---
    '陕西': ['陕西', '陕', '秦', '陕西人'], # 包含双简称
    '甘肃': ['甘肃', '甘', '陇', '甘肃人'], # 包含双简称
    '青海': ['青海', '青', '青海人'],
    '宁夏': ['宁夏', '宁', '宁夏人'],
    '新疆': ['新疆', '新', '新疆人'] # "新"字可能有噪音，但"新疆人"很准
}

# ==============================================================================
# 3. 核心计算函数
# ==============================================================================

def calculate_group_similarity(target_word, group_keywords, model):
    """
    计算 target_word 和 group_keywords 中所有词的相似度，然后取平均。
    实现你要求的：{‘豫’，“河南”，“河南人”} 分别计算 cos 相似度，再平均。
    """
    if target_word not in model:
        return None  # 如果常用词本身不在模型里，跳过
    
    scores = []
    for key in group_keywords:
        if key in model:
            # 计算余弦相似度
            sim = model.similarity(target_word, key)
            scores.append(sim)
    
    # 如果这个省份的所有关键词都不在模型里（极少见），返回 None
    if not scores:
        return None
        
    return np.mean(scores)

# ==============================================================================
# 4. 执行批量计算
# ==============================================================================

# 过滤：先确保常用的词都在模型里，减少循环中的判断
valid_words = [w for w in word_list if w in wv]
print(f"在词向量中找到的有效词汇数量: {len(valid_words)}")

# 用于存储结果的字典列表
results = []

print("正在计算相似度矩阵...")
# 使用 tqdm 显示进度条
for word in tqdm(valid_words):
    row_data = {'word': word}
    
    for prov_name, keywords in province_definitions.items():
        # 调用上面的函数计算平均相似度
        avg_sim = calculate_group_similarity(word, keywords, wv)
        
        # 存入数据，保留4位小数
        if avg_sim is not None:
            row_data[prov_name] = round(float(avg_sim), 4)
        else:
            row_data[prov_name] = np.nan # 或者 0
            
    results.append(row_data)

# ==============================================================================
# 5. 生成表格并保存
# ==============================================================================

# 转换为 DataFrame
df_result = pd.DataFrame(results)

# 设置 'word' 为索引，这样表格看起来就是：行=词，列=省份
df_result.set_index('word', inplace=True)

print("\n计算完成！前 5 行预览：")
print(df_result.head())

# 保存结果
output_file = 'province_similarity_matrix.csv'
df_result.to_csv(output_file, encoding='utf_8_sig') # sig 防止中文乱码
print(f"\n结果已保存至: {output_file}")

原始词表大小: 11310
在词向量中找到的有效词汇数量: 11302
正在计算相似度矩阵...


100%|███████████████████████████████████████████████████████████████████████████| 11302/11302 [00:50<00:00, 225.39it/s]



计算完成！前 5 行预览：
          北京      上海      天津      重庆      河北      山西     内蒙古      东北      辽宁  \
word                                                                           
阿谀奉承  0.1784  0.1455  0.1921  0.1981  0.1779  0.2280  0.1843  0.2521  0.1786   
哀号    0.1150  0.1257  0.1241  0.1572  0.1610  0.1581  0.1798  0.1462  0.1438   
哀求    0.1388  0.1129  0.1607  0.1592  0.1913  0.2034  0.2055  0.1531  0.1744   
哀伤    0.1447  0.1331  0.1642  0.1685  0.1931  0.2042  0.2109  0.1723  0.1862   
哀痛    0.1063  0.0989  0.1174  0.1634  0.1899  0.1960  0.1753  0.1152  0.1748   

          吉林  ...      澳门      四川      贵州      云南      西藏      陕西      甘肃  \
word          ...                                                           
阿谀奉承  0.1572  ...  0.1244  0.2110  0.1879  0.2031  0.1911  0.2030  0.1824   
哀号    0.1447  ...  0.0807  0.1951  0.1440  0.2070  0.1861  0.1749  0.2227   
哀求    0.1724  ...  0.1223  0.2035  0.1637  0.1957  0.1564  0.2421  0.2257   
哀伤    0.2173  ...  0.1549  0.2349  0.16

### 找出省份最相似traits
>计算每个省份最相似的10个traits词汇。再分正面负面找 10个最相似traits 词汇

In [49]:
valence = pd.read_csv('13428_2021_1607_MOESM1_ESM.csv')

In [51]:
valence.columns

Index(['Word_ID', 'Word', 'Valence_Mean', 'Valence_SD', 'Valence_N',
       'Arousal_Mean', 'Arousal_SD', 'Arousal_N', 'Men_Valence_Mean',
       'Men_Valence_SD', 'Men_Valence_N', 'Women_Valence_Mean',
       'Women_Valence_SD', 'Women_Valence_N', 'Men_Arousal_Mean',
       'Men_Arousal_SD', 'Men_Arousal_N', 'Women_Arousal_Mean',
       'Women_Arousal_SD', 'Women_Arousal_N'],
      dtype='object')

In [53]:
valence = valence[['Word', 'Valence_Mean']]

In [55]:
valence

,Word,Valence_Mean
0,阿谀奉承,-2.15
1,哀号,-1.84
2,哀求,-1.69
3,哀伤,-1.85
4,哀痛,-1.75
...,...,...
11305,座右铭,1.28
11306,做东,0.83
11307,做梦,0.15
11308,做主,0.77


In [59]:
import pandas as pd

# ================= 1. 准备 Valence 数据 =================
# 假设 valence 数据文件名为 'valence.csv'，包含 'Word' 和 'Valence_Mean' 列
# 制作字典映射：{'词汇': 分数}，查找速度最快
val_df = valence.copy()
valence_map = val_df.set_index('Word')['Valence_Mean'].to_dict()

# ================= 2. 读取矩阵并预处理 =================
# 读取列表和矩阵
with open("final_trait_list.txt", "r", encoding="utf-8") as f:
    final_list = f.read().splitlines()

# 读取相似度矩阵
df = pd.read_csv('province_similarity_matrix.csv', encoding='utf-8-sig', index_col=0)

# 【关键步骤】
# 1. 筛选行：必须在 trait 列表中，且必须在 wv 模型中（你的代码中有这一步）
# 2. 映射 Valence：直接给矩阵加一列 'valence'，方便后续筛选
valid_traits = [w for w in final_list if w in wv] # 确保 wv 在上下文中存在
df = df[df.index.isin(valid_traits)].copy()       # 过滤行
df['valence'] = df.index.map(valence_map)         # 匹配效价分数

# 去除没有 valence 数据的词（可选，为了计算平均值准确）
df = df.dropna(subset=['valence']) 

# ================= 3. 核心计算函数 =================
def analyze_province(col_series):
    """
    对每个省份的一列数据进行分析
    col_series.name 是省份名，col_series.index 是词汇
    注意：此时 col_series 包含 'valence' 列，需要排除
    """
    # 获取该列对应的词汇相似度（排除 valence 列）
    sim_scores = col_series
    
    # --- A. 总体 Top 10 及其平均 Valence ---
    top10_general = sim_scores.nlargest(10).index.tolist()
    # 通过词汇列表查所有的 valence 并求平均
    avg_valence = df.loc[top10_general, 'valence'].mean()
    
    # --- B. 正向 Top 10 (Valence > 0) ---
    # 利用 df['valence'] 进行布尔筛选，再找该省相似度最高的
    pos_words = sim_scores[df['valence'] > 0].nlargest(10).index.tolist()
    
    # --- C. 负向 Top 10 (Valence < 0) ---
    neg_words = sim_scores[df['valence'] < 0].nlargest(10).index.tolist()
    
    return top10_general, avg_valence, pos_words, neg_words

# ================= 4. 执行并保存 =================
results_general = {}
results_avg_val = {}
results_pos = {}
results_neg = {}

# 遍历省份列（排除最后一列 'valence'）
province_cols = [c for c in df.columns if c != 'valence']

for province in province_cols:
    t10, avg, pos10, neg10 = analyze_province(df[province])
    
    results_general[province] = t10
    results_avg_val[province] = avg
    results_pos[province] = pos10
    results_neg[province] = neg10

# --- 转换格式并保存 ---

# 1. 总体 Top 10 表 (并在最后一行追加平均 Valence)
df_res_general = pd.DataFrame(results_general)
# 追加一行平均值
df_res_general.loc['Mean_Valence'] = pd.Series(results_avg_val)
df_res_general.to_csv("result_top10_general_with_mean.csv", encoding="utf-8-sig")

# 2. 正向 Top 10 表
pd.DataFrame(results_pos).to_csv("result_top10_positive.csv", encoding="utf-8-sig")

# 3. 负向 Top 10 表
pd.DataFrame(results_neg).to_csv("result_top10_negative.csv", encoding="utf-8-sig")

print("分析完成！已生成三个文件：总体(含均值)、正向Top10、负向Top10。")

分析完成！已生成三个文件：总体(含均值)、正向Top10、负向Top10。


In [61]:
df_res_general

,北京,上海,天津,重庆,河北,山西,内蒙古,东北,辽宁,吉林,...,澳门,四川,贵州,云南,西藏,陕西,甘肃,青海,宁夏,新疆
0,可靠,歧视,纯正,豪爽,本分,豪爽,豪爽,豪爽,豪爽,好客,...,乐天,豪爽,好客,豪爽,淳朴,张扬,寒微,豪爽,豪爽,纯正
1,卓越,卓越,老实,好客,歧视,骁勇,纯正,好客,歧视,豪爽,...,歧视,豪迈,豪爽,逍遥,纯朴,豪爽,豪爽,好客,豪迈,野蛮
2,纯正,讨厌,仁爱,仁爱,志愿,本分,好客,豪放,专一,纯正,...,卓越,好客,小气,好客,虔诚,豪迈,张扬,淳朴,卓越,卓越
3,诚信,诚信,博爱,张扬,老实,耿直,歧视,歧视,志愿,文静,...,纯正,仁义,奢侈,张扬,好客,老实,骁勇,逍遥,文静,独立
4,老实,势利,口才,健谈,豪爽,豪迈,老实,豪迈,骁勇,诚实,...,友好,老实,黑心,豪迈,慈悲,骁勇,豪迈,豪迈,仁爱,歧视
5,歧视,博爱,乐天,吹牛,仁爱,牛脾气,淳朴,淳朴,友好,老实,...,诚信,耿直,纯正,文静,豪放,仁义,老实,纯正,好客,好客
6,乐天,小气,吹牛,豪迈,能干,忠烈,豪放,老实,本分,友好,...,信誉,豪放,淳朴,淳朴,豪爽,文静,志气,纯朴,博爱,淳朴
7,万恶,万恶,可靠,博爱,骁勇,博学,豪迈,耿直,文静,本分,...,怀旧,忠义,可靠,爽快,奔放,混账,志向,张扬,老实,老实
8,信誉,挑剔,歧视,老实,刁民,占便宜,志愿,吹牛,豪迈,能干,...,好客,专一,老实,老实,纯正,英明,豪放,文静,纯正,能干
9,鄙视,纯正,友好,文静,专一,渊博,野蛮,纯正,博学,歧视,...,拜金,逍遥,好学,君子,纯洁,流氓,安分,豪放,混账,优秀


### 3.计算省份间两两对比

In [1]:
import pandas as pd
from itertools import permutations

# ================= 1. 数据准备与对齐 =================

# 1.1 读取 Valence 数据，建立查找表
val_df = pd.read_csv('13428_2021_1607_MOESM1_ESM.csv')[['Word', 'Valence_Mean']]
# 假设列名是 'Word' 和 'Valence_Mean'
valence_map = val_df.set_index('Word')['Valence_Mean'].to_dict()

# 1.2 读取相似度矩阵
df_sim = pd.read_csv('province_similarity_matrix.csv', encoding='utf-8-sig', index_col=0)

# 1.3 读取你的 trait 列表
with open("final_trait_list.txt", "r", encoding="utf-8") as f:
    final_list = f.read().splitlines()

# 1.4 【关键过滤】求交集：必须在 trait列表里，且在 valence 库里有分数的词
# 注意：这里假设 df_sim 里的词已经在 wv 里检查过了，如果没检查，这里加上 if w in wv
valid_words = final_list

# 1.5 重构矩阵：只保留有效词，并新增一列 valence 用于后续计算
df_clean = df_sim.loc[valid_words].copy()
df_clean['valence_score'] = df_clean.index.map(valence_map)

print(f"最终参与计算的有效词汇数: {len(df_clean)}")

# ================= 2. 两两对比计算 (Pairwise Comparison) =================

results = []
provinces = [col for col in df_clean.columns if col != 'valence_score']

# 使用 permutations 生成排列：(北京, 上海) 和 (上海, 北京) 都会生成
for p_a, p_b in permutations(provinces, 2):
    
    # --- 核心算法：相对相似度 = A的相似度 - B的相似度 ---
    # 直接整列相减，Pandas 会自动对齐索引
    diff_series = df_clean[p_a] - df_clean[p_b]
    
    # 找出差值最大的 Top 10 (即 A 相比 B 最突出的词)
    top10_words = diff_series.nlargest(10).index.tolist()
    
    # 计算这 10 个词的平均 Valence
    # 直接从 df_clean 里查这些词的 valence_score 列
    avg_val = df_clean.loc[top10_words, 'valence_score'].mean()
    
    # 格式化 Top 10 词汇为字符串 "词1, 词2, ..."
    traits_str = ", ".join(top10_words)
    
    # 存入结果
    results.append({
        'Group A': p_a,
        'vs. Group B': p_b,
        'Top 10 Relative Traits': traits_str,
        'Average Valence': round(avg_val, 3) # 保留3位小数
    })

# ================= 3. 保存结果 =================

df_result = pd.DataFrame(results)

# 按照你想要的格式列出
# 比如你想看 北京 vs 其他所有省，可以按 Group A 排序
df_result = df_result.sort_values(by=['Group A', 'vs. Group B'])

print(df_result.head())
df_result.to_csv("province_pairwise_comparison.csv", encoding='utf-8-sig', index=False)
print("计算完成，结果已保存至 province_pairwise_comparison.csv")

最终参与计算的有效词汇数: 799
   Group A vs. Group B                       Top 10 Relative Traits  \
40      上海          东北       卓越, 健忘, 周到, 准时, 细致, 审慎, 锐意, 拖延, 可靠, 完美   
61      上海          云南    泼妇, 诚信, 倚老卖老, 素质, 势利, 怀旧, 人情味, 歧视, 拜金, 外向   
39      上海         内蒙古  拜金, 人情味, 从良, 倚老卖老, 势利, 刻薄, 泼妇, 乐天, 挑剔, 八面玲珑   
34      上海          北京     念旧, 激进, 泼妇, 拜金, 同情心, 陋习, 风骚, 痴心, 求知欲, 怪僻   
50      上海          台湾     挑剔, 急躁, 恒心, 细心, 淘气, 泼妇, 机智, 絮叨, 洒脱, 体贴入微   

    Average Valence  
40            0.955  
61           -0.249  
39           -0.571  
34           -0.443  
50            0.277  
计算完成，结果已保存至 province_pairwise_comparison.csv
